Confirming the Clustering Results of Labels A and B with GISAID Labels.

In [ ]:
import pandas as pd
import numpy as np
import math
import csv

from matplotlib import pyplot as plt

In [2]:
#All the removed files
re=['EPI_ISL_402131.fa', 'EPI_ISL_408514.fa', 'EPI_ISL_408515.fa', 'EPI_ISL_410721.fa', 'EPI_ISL_417024.fa', 'EPI_ISL_418206.fa', 'EPI_ISL_418207.fa', 'EPI_ISL_418208.fa', 'EPI_ISL_418209.fa', 'EPI_ISL_418796.fa', 'EPI_ISL_418810.fa', 'EPI_ISL_419529.fa', 'EPI_ISL_419530.fa', 'EPI_ISL_419531.fa', 'EPI_ISL_419532.fa', 'EPI_ISL_419533.fa', 'EPI_ISL_419534.fa', 'EPI_ISL_419535.fa', 'EPI_ISL_419536.fa', 'EPI_ISL_419537.fa', 'EPI_ISL_419538.fa', 'EPI_ISL_419539.fa', 'EPI_ISL_419540.fa', 'EPI_ISL_419561.fa']

In [20]:
alllabels = []
for thing in data3:
    for thi in thing:
        alllabels.append(thi)


# Accuracy Metrics

In [21]:
labelz = ["A", "B"]

#Label clusters into group A and B based on length - A is the smaller group
def dividedata(test):
    testlabels = {}
    testlabels.update({"A":test[0]})
    testlabels.update({"B":test[1]})
    if len(test[1]) < len(testlabels["A"]):
        testlabels.update({"A":test[1]})
        testlabels.update({"B":test[0]})
    return testlabels
    
#Output array as [TP, FN, FP, and TN]
def return4cells(testlabels, reallabels):
    labeindex = 0
    summetrics = {}
    for thing in labelz: 
        blar = []
        TP = 0
        for i in testlabels[thing]:
            if i in reallabels[thing]:
                TP +=1
        blar.append(TP)
        FN = 0
        for i in reallabels[thing]:
            if i not in testlabels[thing]:
                FN +=1
        blar.append(FN)
        FP = 0
        for i in testlabels[thing]:
            if i not in reallabels[thing]:
                FP +=1
        blar.append(FP)
        TN = 0
        if labeindex == 0:
            for i in testlabels["B"]:
                if i in reallabels["B"]:
                    TN +=1
        if labeindex == 1:
            for i in testlabels["A"]:
                if i in reallabels["A"]:
                    TN +=1
        blar.append(TN)
        summetrics.update({thing:blar})
    return summetrics

def precision(outputcells):
    return (outputcells[0]/(outputcells[0]+outputcells[2]))

def truepostive(outputcells):
    return (outputcells[0]/(outputcells[0]+outputcells[1]))

def falsepositive(outputcells):
    return (outputcells[2]/(outputcells[2]+outputcells[3]))

### Testing Functions Above

In [22]:
bleb2 = dividedata(data3)
belb2 = return4cells(bleb2, gisaidsorted)
print(type(belb2["A"]))

<class 'list'>


In [23]:
aas = falsepositive(belb2["A"])
print(aas)

1.0


In [24]:
print(len(gisaidsorted["A"]))
print(len(gisaidsorted["B"]))

0
0


# GISAID Labels

In [26]:
f = open("GISAID_classification.txt")
stuff = f.read()
splitstuff = stuff.splitlines(True)

In [27]:
splitstuff[0]

'>USA/WA-S88/EPI_ISL_417141/2020-03-01/S/A.1\n'

In [28]:
alllabels[0]

'EPI_ISL_402119.fa'

In [29]:
gisadlabels  = {}
for line in splitstuff:
    bar = line.split("/")
    if bar[2]+".fa" in alllabels:
        gisadlabels.update({bar[2]+".fa":bar[5]})
    #elif bar[2]+."fa"

In [30]:
print(len(gisadlabels))

3946


In [31]:
gisaidsorted = {}
labels = ["A","B"]
for i in labels:
    labelstuff = []
    for thing in gisadlabels:
        blue = gisadlabels[thing]
        if blue[0] == i:
            labelstuff.append(thing)
    gisaidsorted.update({i:labelstuff}) 

In [32]:
print(len(gisaidsorted["A"]))
print(len(gisaidsorted["B"]))

727
3219


# Model Outputs

In [33]:
with open('L1_k25_kmeans_c2.csv', newline='\n') as f:
    reader1 = csv.reader(f)
    data1 = list(reader1)

with open("L2_k25_kmeans_c2.csv", newline="\n") as f:
    reader2 = csv.reader(f)
    data2 = list(reader2)

with open('jaccard_k25_kmeans_c2.csv', newline="\n") as f:
    reader3 = csv.reader(f)
    data3 = list(reader3)
    
with open("msa_kmeans_c2.csv", newline="\n") as f:
    reader4 = csv.reader(f)
    data4 = list(reader4)

In [34]:
allcompiled = []
allcompiled.append(data1)
allcompiled.append(data2)
allcompiled.append(data3)
allcompiled.append(data4)

allmetrics = []
for datta in allcompiled:
    eep = dividedata(datta)
    stuffto = return4cells(eep, gisaidsorted)
    allmetrics.append(stuffto)
    
alltpstuff = []
for datta in allmetrics:
    yo = {}
    for ep in datta:
        yum = datta[ep]
        tp = precision(yum)
        yy = falsepositive(yum)
        yyy = truepostive(yum)
        yo.update({ep:[tp,yy,yy]})
    alltpstuff.append(yo)

In [35]:
alltpstuff

[{'A': [0.4555984555984556, 0.26281453867660765, 0.26281453867660765],
  'B': [0.992056856187291, 0.00794314381270903, 0.00794314381270903]},
 {'A': [0.24923857868020305, 0.4594594594594595, 0.4594594594594595],
  'B': [0.8805668016194332, 0.1194331983805668, 0.1194331983805668]},
 {'A': [0.4553054662379421, 0.26312519415967695, 0.26312519415967695],
  'B': [0.9920535340861564, 0.00794646591384358, 0.00794646591384358]},
 {'A': [0.15601023017902813, 0.10251630941286113, 0.10251630941286113],
  'B': [0.8126582278481013, 0.18734177215189873, 0.18734177215189873]}]

# Testing with Sklearn Functions

In [36]:
import numpy as np
from sklearn.metrics import average_precision_score
y_true = np.array([0, 0, 1, 1])
y_scores = np.array([0.1, 0.4, 0.35, 0.8])
average_precision_score(y_true, y_scores)

0.8333333333333333

In [37]:
test1 = []
oop = {"A":0, "B":1}
for thing in bleb2:
    for j in bleb2[thing]:
        test1.append(oop[thing])

truetest = []
for thing in gisaidsorted:
    for j in gisaidsorted[thing]:
        truetest.append(oop[thing])

In [197]:
import numpy as np
from sklearn.metrics import precision_recall_curve
y_true = np.array(truetest)
y_scores = np.array(test1)
precision, recall, thresholds = precision_recall_curve(y_true, y_scores)

In [203]:
y_true

array([0, 0, 0, ..., 1, 1, 1])

In [207]:
print(alllabels[3943])

EPI_ISL_419424.fa


In [212]:
yo = alllabels[0]
yoo = float(yo[8:-3])
for thing in alllabels:
    if float(thing[8:-3]) > yoo:
        yoo = float(thing[8:-3])
    #print(thing[8:-3])

In [213]:
yoo

422186.0